In [1]:
import cv2
import joblib
import mediapipe as mp
import numpy as np

# Load the trained model
model_path = "../Dataset/Generated_Data/mlp_classifier_best_params.pkl"
mlp_model = joblib.load(model_path)

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Load class labels
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
                'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'space']  # Adjust based on your dataset labels

def extract_hand_landmarks(image):
    """Extracts 21 hand landmarks from the given image."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        data = []
        for landmark in landmarks.landmark:
            data.append(landmark.x)
            data.append(landmark.y)
        return np.array(data).reshape(1, -1)  # Reshape for model input
    return None

# Open webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip image for natural hand positioning
    frame = cv2.flip(frame, 1)

    # Extract hand landmarks
    landmarks = extract_hand_landmarks(frame)

    if landmarks is not None:
        # Make prediction
        prediction = mlp_model.predict(landmarks)
        predicted_label = prediction[0]

        # Draw label on the frame
        cv2.putText(frame, f'Prediction: {predicted_label}', (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the video feed
    cv2.imshow("ASL Hand Sign Prediction", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


I0000 00:00:1740536818.477041 18090294 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M4 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740536818.485120 18090567 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740536818.491093 18090567 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-02-25 18:26:59.574 python[53905:18090294] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 18:26:59.574 python[53905:18090294] +[IMKInputSession subclass]: chose IMKInputSession_Modern
W0000 00:00:1740536819.945755 18090573 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
